In [84]:
import pandas as pd
data = pd.read_excel('turkish_topic_modelling.xlsx')
#data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['Document']]
data_text['index'] = data_text.index
documents = data_text

In [85]:
print(len(documents))
print(documents[:5])

647809
                                            Document  index
0  NGC 5713 Başak takımyıldızı bölgesinde bulunan...      0
1  Birçok katalogda sarmal gökada olarak sınıflan...      1
2  Corina Casanova , İsviçre Federal Şansölyesidir .      2
3  Casanova , İsviçre Federal Yüksek Mahkemesi es...      3
4       Corina Casanova bir federal parlementerdir .      4


In [86]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [93]:
"""!pip install -q wordcloud
!pip install nltk
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') """
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import io
import unicodedata
import numpy as np
import re
import string
from nltk.stem.snowball import SnowballStemmer
#snow_stemmer = SnowballStemmer(language='english')
from snowballstemmer import TurkishStemmer
snow_stemmer=TurkishStemmer()
"""from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
porter = PorterStemmer()
lancaster=LancasterStemmer()"""
def lemmatize_stemming(text):
    return snow_stemmer.stemWord(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [94]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
preprocess(doc_sample)

original document: 
['Deri', 'yapısının', 'bozunabilirliği', 'kıllanımında', 'ciddi', 'problemlere', 'yol', 'açarken', 'kalıcı', 'dayanıklılık', 've', 'aynı', 'zamanda', 'kullanılabilirlik', 'sağlamada', 'temel', 'aşama', 'tabaklamadır', '.']


 tokenized and lemmatized document: 


['der',
 'yapı',
 'bozunabilirlik',
 'kıllanım',
 'cidi',
 'problem',
 'açar',
 'kalıç',
 'dayanıklılık',
 'aynı',
 'zama',
 'sağlama',
 'temel',
 'aşa',
 'tabakla']

In [96]:
processed_docs = documents['Document'].map(preprocess)
processed_docs[:10]

0    [başak, takımyıldız, bölge, buluna, tuhaf, asi...
1    [birçok, katalogu, sarmal, göka, olarak, sınıf...
2           [cor, casanov, sviçre, federal, şansölyes]
3    [casanov, sviçre, federal, yük, mahkemes, eski...
4                 [cor, casanov, federal, parlementer]
5    [casanov, hristiya, demokrat, halk, partis, üy...
6    [sviçre, dışiş, bakanlık, sviçre, federal, yön...
7                 [sviçre, ilişki, sürdürmek, görevli]
8    [gilgit, baltis, veya, pakis, kuzey, bölge, ur...
9       [ala, kaplamak, oldukça, dağlık, arazi, sahip]
Name: Document, dtype: object

In [97]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 asimetrik
1 başak
2 buluna
3 bölge
4 göka
5 takımyıldız
6 tuhaf
7 birçok
8 farklı
9 gökada
10 katalogu


In [98]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [99]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(129, 1),
 (133, 1),
 (141, 1),
 (428, 1),
 (458, 1),
 (499, 1),
 (1969, 1),
 (2879, 1),
 (3281, 1),
 (3372, 1),
 (5020, 1),
 (7033, 1)]

In [100]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 129 ("yapı") appears 1 time.
Word 133 ("aynı") appears 1 time.
Word 141 ("zama") appears 1 time.
Word 428 ("dayanıklılık") appears 1 time.
Word 458 ("temel") appears 1 time.
Word 499 ("cidi") appears 1 time.
Word 1969 ("der") appears 1 time.
Word 2879 ("aşa") appears 1 time.
Word 3281 ("kalıç") appears 1 time.
Word 3372 ("açar") appears 1 time.
Word 5020 ("sağlama") appears 1 time.
Word 7033 ("problem") appears 1 time.


In [101]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5317289678349512),
 (1, 0.4553130927055291),
 (2, 0.20012217028669313),
 (3, 0.19410818121482892),
 (4, 0.3678576477058213),
 (5, 0.3373613792126616),
 (6, 0.427896569466722)]


In [102]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [103]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"bölge" + 0.019*"ola" + 0.019*"bağlı" + 0.018*"merkez" + 0.017*"kuzey" + 0.015*"buluna" + 0.015*"ilçe" + 0.014*"güney" + 0.014*"eyalet" + 0.014*"il"
Topic: 1 
Words: 0.023*"sonra" + 0.015*"iç" + 0.011*"ola" + 0.011*"çocuk" + 0.010*"yaş" + 0.010*"kendi" + 0.009*"dah" + 0.009*"olduk" + 0.007*"kardeş" + 0.007*"oğlu"
Topic: 2 
Words: 0.021*"savaş" + 0.012*"yıl" + 0.011*"sonra" + 0.010*"taraf" + 0.009*"olarak" + 0.009*"karşı" + 0.008*"birlik" + 0.008*"iç" + 0.008*"alma" + 0.008*"osmanlı"
Topic: 3 
Words: 0.046*"yıl" + 0.014*"sonra" + 0.012*"olarak" + 0.012*"üniversites" + 0.011*"yap" + 0.010*"ol" + 0.008*"okul" + 0.008*"kadar" + 0.008*"al" + 0.007*"eğit"
Topic: 4 
Words: 0.052*"film" + 0.019*"yap" + 0.015*"filmi" + 0.014*"oy" + 0.010*"karakter" + 0.008*"televizyo" + 0.008*"sinema" + 0.008*"oyuncu" + 0.008*"ola" + 0.007*"adlı"
Topic: 5 
Words: 0.024*"tak" + 0.022*"futbol" + 0.020*"maç" + 0.015*"ol" + 0.015*"yıl" + 0.014*"ligi" + 0.014*"takım" + 0.013*"kupas" + 0.013*"s

In [104]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.004*"olarak" + 0.004*"iç" + 0.003*"yıl" + 0.003*"ola" + 0.003*"üzer" + 0.002*"istatistik" + 0.002*"renk" + 0.002*"taraf" + 0.002*"büyük" + 0.002*"selçuklu"
Topic: 1 Word: 0.011*"üniversites" + 0.009*"yıl" + 0.005*"schleswigi" + 0.005*"okul" + 0.005*"eğit" + 0.005*"holstein" + 0.004*"sonra" + 0.004*"ödül" + 0.004*"olarak" + 0.004*"mez"
Topic: 2 Word: 0.011*"bölge" + 0.011*"il" + 0.010*"eyalet" + 0.010*"bağlı" + 0.010*"ilçe" + 0.009*"kuzey" + 0.008*"şehir" + 0.008*"merkez" + 0.007*"güney" + 0.006*"buluna"
Topic: 3 Word: 0.013*"tak" + 0.012*"maç" + 0.011*"futbol" + 0.009*"ligi" + 0.009*"takım" + 0.008*"kupas" + 0.008*"yıl" + 0.008*"kulüp" + 0.008*"sezo" + 0.007*"ol"
Topic: 4 Word: 0.004*"ola" + 0.004*"yıl" + 0.004*"taraf" + 0.003*"olarak" + 0.003*"vilayet" + 0.003*"oğlu" + 0.003*"sonra" + 0.002*"türk" + 0.002*"savaş" + 0.002*"kral"
Topic: 5 Word: 0.005*"olarak" + 0.005*"iç" + 0.004*"japo" + 0.004*"eski" + 0.004*"gör" + 0.003*"olduk" + 0.003*"gip" + 0.003*"ola" + 0.003*"da

In [105]:
processed_docs[4310]

['der',
 'yapı',
 'bozunabilirlik',
 'kıllanım',
 'cidi',
 'problem',
 'açar',
 'kalıç',
 'dayanıklılık',
 'aynı',
 'zama',
 'sağlama',
 'temel',
 'aşa',
 'tabakla']

In [106]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6829941868782043	 
Topic: 0.024*"iç" + 0.024*"olarak" + 0.010*"zama" + 0.008*"üzer" + 0.008*"dah" + 0.007*"olduk" + 0.007*"gip" + 0.006*"veya" + 0.006*"özellik" + 0.006*"gör"

Score: 0.1689678281545639	 
Topic: 0.023*"sonra" + 0.015*"iç" + 0.011*"ola" + 0.011*"çocuk" + 0.010*"yaş" + 0.010*"kendi" + 0.009*"dah" + 0.009*"olduk" + 0.007*"kardeş" + 0.007*"oğlu"

Score: 0.09418332576751709	 
Topic: 0.021*"savaş" + 0.012*"yıl" + 0.011*"sonra" + 0.010*"taraf" + 0.009*"olarak" + 0.009*"karşı" + 0.008*"birlik" + 0.008*"iç" + 0.008*"alma" + 0.008*"osmanlı"


In [107]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.8064629435539246	 
Topic: 0.005*"olarak" + 0.005*"iç" + 0.004*"japo" + 0.004*"eski" + 0.004*"gör" + 0.003*"olduk" + 0.003*"gip" + 0.003*"ola" + 0.003*"dah" + 0.003*"veya"

Score: 0.1319911628961563	 
Topic: 0.005*"iç" + 0.004*"olduk" + 0.003*"olarak" + 0.003*"kendi" + 0.003*"sonra" + 0.003*"ola" + 0.003*"dah" + 0.002*"turnuva" + 0.002*"çocuk" + 0.002*"baş"


In [115]:
unseen_document = 'Joel bu durumu şöyle açıkladı : " Amacımız daha çok , otelin John Goodman\'ın oynadığı karakterin dışsallaştırılması görevini üstlenmesiydi . teri tıpkı duvar kağıdının soyulup '
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.35681307315826416	 Topic: 0.023*"sonra" + 0.015*"iç" + 0.011*"ola" + 0.011*"çocuk" + 0.010*"yaş"
Score: 0.22342631220817566	 Topic: 0.052*"film" + 0.019*"yap" + 0.015*"filmi" + 0.014*"oy" + 0.010*"karakter"
Score: 0.22060909867286682	 Topic: 0.024*"iç" + 0.024*"olarak" + 0.010*"zama" + 0.008*"üzer" + 0.008*"dah"
Score: 0.10388186573982239	 Topic: 0.021*"savaş" + 0.012*"yıl" + 0.011*"sonra" + 0.010*"taraf" + 0.009*"olarak"
Score: 0.06748420745134354	 Topic: 0.025*"bölge" + 0.019*"ola" + 0.019*"bağlı" + 0.018*"merkez" + 0.017*"kuzey"
